# Import libraries

In [1]:
import kagglehub
import pandas as pd
from kagglehub import KaggleDatasetAdapter

/Users/Delta/opt/anaconda3/envs/dsi_participant/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load the data into a Panda dataframe

In [2]:
df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    'mattiuzc/stock-exchange-data',
    path='indexData.csv')

df.head()

,Index,Date,Open,High,Low,Close,Adj Close,Volume
0,NYA,1965-12-31,528.690002,528.690002,528.690002,528.690002,528.690002,0.0
1,NYA,1966-01-03,527.210022,527.210022,527.210022,527.210022,527.210022,0.0
2,NYA,1966-01-04,527.840027,527.840027,527.840027,527.840027,527.840027,0.0
3,NYA,1966-01-05,531.119995,531.119995,531.119995,531.119995,531.119995,0.0
4,NYA,1966-01-06,532.070007,532.070007,532.070007,532.070007,532.070007,0.0


# Filter data based on index and date range

In [3]:
# Filter rows where Index is NYA, IXIC or GSPTSE

df = df[df['Index'].isin(['NYA', 'IXIC', 'GSPTSE'])]

# Convert the Date column from object to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Split up the dataset into pre-covid / post-covid time frames
precovid_start_date = '2018-01-01'
precovid_end_date = '2020-02-29'

post_covid_start_date = '2020-03-01'
post_covid_end_date = '2021-05-31'

precovid_df = df[(df['Date'] >= precovid_start_date) & (df['Date'] <= precovid_end_date)]
postcovid_df = df[(df['Date'] >= post_covid_start_date) & (df['Date'] <= post_covid_end_date)]

print(f"precovid_df min date = {precovid_df['Date'].min()}, precovid_df max date = {precovid_df['Date'].max()}")
print(f"post_covid_df min date = {postcovid_df['Date'].min()}, postcovid_df max date = {postcovid_df['Date'].max()}")

precovid_df min date = 2018-01-02 00:00:00, precovid_df max date = 2020-02-28 00:00:00
post_covid_df min date = 2020-03-02 00:00:00, postcovid_df max date = 2021-05-31 00:00:00


# Check if there are any missing / nan values

In [4]:
precovid_df.isnull().sum()

Index        0
Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [5]:
postcovid_df.isnull().sum()

Index        0
Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

# Sort by Index and Date

In [6]:
precovid_df = precovid_df.sort_values(['Index', 'Date']).reset_index(drop=True)
postcovid_df = postcovid_df.sort_values(['Index', 'Date']).reset_index(drop=True)

In [7]:
precovid_df.head()

,Index,Date,Open,High,Low,Close,Adj Close,Volume
0,GSPTSE,2018-01-02,16213.40039,16310.00000,16181.00000,16310.00000,16310.00000,1.570734e+10
1,GSPTSE,2018-01-03,16336.70020,16386.30078,16322.59961,16371.59961,16371.59961,1.968654e+10
2,GSPTSE,2018-01-04,16385.40039,16421.40039,16344.90039,16412.90039,16412.90039,1.959752e+10
3,GSPTSE,2018-01-05,16362.59961,16370.29981,16309.90039,16349.40039,16349.40039,1.603131e+10
4,GSPTSE,2018-01-08,16355.40039,16373.50000,16297.90039,16317.70020,16317.70020,1.660954e+10


In [8]:
print(postcovid_df.columns)

Index(['Index', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')


# Add rows for days when the market is closed (e.g. weekends, holidays) and Market Closed column

In [9]:
def add_market_closure_rows(df):
    # Compute full date range from min to max date (e.g. 2019-03-01 to 2020-02-29)
    date_range = pd.date_range(start=df['Date'].min(), end=df['Date'].max(), freq='D')

    # Add new rows to the data frame (all columns will be NaN, including Index)
    # This temporarily sets dataframe index to Date (the index will be reset later)
    df = df.set_index('Date').reindex(date_range)

    # Add MarketClosed column
    df['Market Closed'] = df['Open'].isna()

    # Forward-fill columns
    df = df.ffill()

    # Reset dataframe index (not the Index column)
    df = df.reset_index().rename(columns={'index': 'Date'})

    return df

precovid_df = precovid_df.groupby('Index').apply(add_market_closure_rows).reset_index(drop=True)
postcovid_df = postcovid_df.groupby('Index').apply(add_market_closure_rows).reset_index(drop=True)
precovid_df.head(20)


/var/folders/0t/3rqmkkps2tx4pjwf78xvy2xh0000gn/T/ipykernel_11722/2565042700.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  precovid_df = precovid_df.groupby('Index').apply(add_market_closure_rows).reset_index(drop=True)
/var/folders/0t/3rqmkkps2tx4pjwf78xvy2xh0000gn/T/ipykernel_11722/2565042700.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  postcovid_df = postcovid_df.groupby('Index').apply(add_

,Date,Index,Open,High,Low,Close,Adj Close,Volume,Market Closed
0,2018-01-02,GSPTSE,16213.40039,16310.00000,16181.00000,16310.00000,16310.00000,1.570734e+10,False
1,2018-01-03,GSPTSE,16336.70020,16386.30078,16322.59961,16371.59961,16371.59961,1.968654e+10,False
2,2018-01-04,GSPTSE,16385.40039,16421.40039,16344.90039,16412.90039,16412.90039,1.959752e+10,False
3,2018-01-05,GSPTSE,16362.59961,16370.29981,16309.90039,16349.40039,16349.40039,1.603131e+10,False
4,2018-01-06,GSPTSE,16362.59961,16370.29981,16309.90039,16349.40039,16349.40039,1.603131e+10,True
5,2018-01-07,GSPTSE,16362.59961,16370.29981,16309.90039,16349.40039,16349.40039,1.603131e+10,True
6,2018-01-08,GSPTSE,16355.40039,16373.50000,16297.90039,16317.70020,16317.70020,1.660954e+10,False
7,2018-01-09,GSPTSE,16351.40039,16358.20020,16309.79981,16319.20020,16319.20020,2.369117e+10,False
8,2018-01-10,GSPTSE,16314.20020,16341.09961,16229.29981,16248.00000,16248.00000,2.439533e+10,False
9,2018-01-11,GSPTSE,16276.20020,16301.40039,16244.90039,16286.90039,16286.90039,2.311612e+10,False


# Calculate 10-day and 21-day moving averages

In [10]:
precovid_df['Adj Close Moving Average 10'] = precovid_df.groupby('Index')['Adj Close'].transform(lambda x: x.rolling(window=10, min_periods=1).mean())
precovid_df['Adj Close Moving Average 21'] = precovid_df.groupby('Index')['Adj Close'].transform(lambda x: x.rolling(window=21, min_periods=1).mean())

postcovid_df['Adj Close Moving Average 10'] = postcovid_df.groupby('Index')['Adj Close'].transform(lambda x: x.rolling(window=10, min_periods=1).mean())
postcovid_df['Adj Close Moving Average 21'] = postcovid_df.groupby('Index')['Adj Close'].transform(lambda x: x.rolling(window=21, min_periods=1).mean())

In [11]:
precovid_df.head(20)

,Date,Index,Open,High,Low,Close,Adj Close,Volume,Market Closed,Adj Close Moving Average 10,Adj Close Moving Average 21
0,2018-01-02,GSPTSE,16213.40039,16310.00000,16181.00000,16310.00000,16310.00000,1.570734e+10,False,16310.000000,16310.000000
1,2018-01-03,GSPTSE,16336.70020,16386.30078,16322.59961,16371.59961,16371.59961,1.968654e+10,False,16340.799805,16340.799805
2,2018-01-04,GSPTSE,16385.40039,16421.40039,16344.90039,16412.90039,16412.90039,1.959752e+10,False,16364.833333,16364.833333
3,2018-01-05,GSPTSE,16362.59961,16370.29981,16309.90039,16349.40039,16349.40039,1.603131e+10,False,16360.975098,16360.975098
4,2018-01-06,GSPTSE,16362.59961,16370.29981,16309.90039,16349.40039,16349.40039,1.603131e+10,True,16358.660156,16358.660156
5,2018-01-07,GSPTSE,16362.59961,16370.29981,16309.90039,16349.40039,16349.40039,1.603131e+10,True,16357.116862,16357.116862
6,2018-01-08,GSPTSE,16355.40039,16373.50000,16297.90039,16317.70020,16317.70020,1.660954e+10,False,16351.485910,16351.485910
7,2018-01-09,GSPTSE,16351.40039,16358.20020,16309.79981,16319.20020,16319.20020,2.369117e+10,False,16347.450196,16347.450196
8,2018-01-10,GSPTSE,16314.20020,16341.09961,16229.29981,16248.00000,16248.00000,2.439533e+10,False,16336.400174,16336.400174
9,2018-01-11,GSPTSE,16276.20020,16301.40039,16244.90039,16286.90039,16286.90039,2.311612e+10,False,16331.450196,16331.450196


In [12]:
postcovid_df.head(20)

,Date,Index,Open,High,Low,Close,Adj Close,Volume,Market Closed,Adj Close Moving Average 10,Adj Close Moving Average 21
0,2020-03-02,GSPTSE,16325.00000,16566.69922,16166.29981,16553.30078,16553.30078,3.747215e+10,False,16553.300780,16553.300780
1,2020-03-03,GSPTSE,16674.90039,16798.19922,16378.29981,16423.59961,16423.59961,3.931279e+10,False,16488.450195,16488.450195
2,2020-03-04,GSPTSE,16659.90039,16779.50000,16539.40039,16779.50000,16779.50000,2.737186e+10,False,16585.466797,16585.466797
3,2020-03-05,GSPTSE,16541.59961,16684.90039,16456.69922,16554.00000,16554.00000,3.130123e+10,False,16577.600097,16577.600097
4,2020-03-06,GSPTSE,16222.09961,16261.50000,16015.79981,16175.00000,16175.00000,3.747038e+10,False,16497.080078,16497.080078
5,2020-03-07,GSPTSE,16222.09961,16261.50000,16015.79981,16175.00000,16175.00000,3.747038e+10,True,16443.400065,16443.400065
6,2020-03-08,GSPTSE,16222.09961,16261.50000,16015.79981,16175.00000,16175.00000,3.747038e+10,True,16405.057199,16405.057199
7,2020-03-09,GSPTSE,15221.79981,15240.29981,14498.29981,14514.20020,14514.20020,5.592274e+10,False,16168.700074,16168.700074
8,2020-03-10,GSPTSE,15019.00000,15048.79981,14481.90039,14958.09961,14958.09961,5.710013e+10,False,16034.188911,16034.188911
9,2020-03-11,GSPTSE,14639.70020,14714.20020,14185.20020,14270.09961,14270.09961,4.689909e+10,False,15857.779981,15857.779981


# Save results to CSV files

In [13]:
import os
os.makedirs('../data', exist_ok=True)
precovid_df.to_csv('../data/precovid.csv', index=False)
postcovid_df.to_csv('../data/postcovid.csv', index=False)